## Databases 

In [1]:
import os
import pandas as pd
import time

In [2]:
hostname = "127.0.0.1"
dataDir = os.listdir("../data/full/")
dataDir

['.ipynb_checkpoints',
 'hospital-testData.csv',
 'hospital-testData.json',
 'hospital-tab.csv',
 'hospital-tab.json',
 'college-tab.json']

In [3]:
hospital = pd.read_csv("../data/full/hospital-tab.csv").to_dict("records")

In [4]:
college = pd.read_json("../data/full/college-tab.json").to_dict("records")

### Mongo 

In [71]:
from pymongo import MongoClient

In [344]:
client = MongoClient(host=hostname,port=27017)

In [345]:
client

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)

In [ ]:
[[e.update(id=idx+1) for idx, e in enumerate(data)] for data in [hospital, college]]

In [ ]:
hospitalMongo = client["hospital"]["hospital"]
collegeMongo = client["college"]["college"]

In [ ]:
hospitalMongo.insert_many(hospital)
collegeMongo.insert_many(college)

### Neo4j 

In [5]:
hospital[0].keys()
college[0].keys()

dict_keys(['Name', 'Address', 'City', 'Zip', 'Web', 'ProgramName'])

In [347]:
#from neo4j import GraphDatabase

In [6]:
from py2neo import Graph, Node, Relationship

In [7]:
txpool = []  
for e in hospital: 
    start = Node("Hospital",name=e["start_name"],lat=e["start_loc_lat"],lon=e["start_loc_lon"])
    end = Node("Hospital",name=e["end_name"],lat=e["end_loc_lat"],lon=e["end_loc_lon"])
    
    r = Relationship(start,"NEAREST_PATH",end)
        
    txpool.append(start)
    txpool.append(end)
    txpool.append(r)

In [8]:
unique = set([e["Name"] for e in college])

for idx, collegeName in enumerate(unique):
    # Filter 
    pred = list(filter(lambda e: e["Name"] == collegeName, college)) 
    getProgramName = list(map(lambda e: Node("ProgramName",name=e["ProgramName"]), pred))
                              
    university = Node("University", name=pred[0]["Name"], web=pred[0]["Web"], city=pred[0]["City"])    
    
    [txpool.append(Relationship(university,"TEACHES",e)) for e in getProgramName]
    [txpool.append(Relationship(e,"HAS_CAPACITY",university)) for e in getProgramName]

    print(f"{idx} - {collegeName} - {len(pred)} - {len(getProgramName)}")

0 - Medical University of South Carolina - 21 - 21
1 - University of West Georgia - 7 - 7
2 - University of Mount Olive - 1 - 1
3 - Mercer County Technical Schools - Assunpink Center - 1 - 1
4 - Southern University at Shreveport - 4 - 4
5 - VA Pacific Islands Health Care System - 1 - 1
6 - Oakland City University - 2 - 2
7 - Alliant International University - 13 - 13
8 - Hesston College - 1 - 1
9 - The Boston Conservatory - 1 - 1
10 - Belhaven University - 5 - 5
11 - McKendree University - 4 - 4
12 - VA Salt Lake City Health Care System - 4 - 4
13 - Holy Family University - 4 - 4
14 - UCEDA International Paterson - 1 - 1
15 - Yeshiva University - 7 - 7
16 - Western Technology Center - Weatherford - 1 - 1
17 - Mississippi Gulf Coast Community College - 4 - 4
18 - University of West Alabama - 2 - 2
19 - Minneapolis School of Anesthesia - 1 - 1
20 - Howard College - 3 - 3
21 - Hillyard Technical Center - 2 - 2
22 - NorthWest Arkansas Community College - 1 - 1
23 - Texas A&M International 

186 - University of St. Thomas - 9 - 9
187 - Phillips Community College of the University of Arkansas - 1 - 1
188 - William T. McFatter Technical College - 1 - 1
189 - Pima Community College - 9 - 9
190 - Fortis Institute - Scranton - 1 - 1
191 - Tabor College - 3 - 3
192 - Bluegrass Community and Technical College - 6 - 6
193 - Trident Technical College - 7 - 7
194 - University of California, San Francisco - 26 - 26
195 - Everest College - Reseda - 1 - 1
196 - North Hennepin Community College - 1 - 1
197 - Odessa College - 4 - 4
198 - Maranatha Baptist University - 1 - 1
199 - Fisher College - 1 - 1
200 - Drury University - 2 - 2
201 - Ohio Northern University - 5 - 5
202 - Cleveland State University - 13 - 13
203 - Central Community College - 4 - 4
204 - Christian Brothers University - 2 - 2
205 - Salem Community College - 2 - 2
206 - Hurley Medical Center - 2 - 2
207 - Southern Nazarene University - 4 - 4
208 - Southeast Missouri Hospital College of Nursing - 3 - 3
209 - Lynn Univer

451 - Cass Career Center - 1 - 1
452 - University of Saint Mary - 4 - 4
453 - Pikes Peak Community College - 2 - 2
454 - Southern State Community College - 1 - 1
455 - Life University - 3 - 3
456 - Laguna College of Art and Design - 1 - 1
457 - Governors State University - 8 - 8
458 - Saint John's Seminary - 1 - 1
459 - University of Hawaii at Hilo - 5 - 5
460 - Keene State College - 5 - 5
461 - Boonslick Technical Education Center - 1 - 1
462 - Saint Catharine College - 4 - 4
463 - Augustana University - 5 - 5
464 - Northeastern Oklahoma Psychology Internship Program - 1 - 1
465 - York Technical Institute - 1 - 1
466 - University of Louisiana at Monroe - 12 - 12
467 - Carl Sandburg College - 5 - 5
468 - Terra State Community College - 2 - 2
469 - Crawford County Area Vocational Technical School Practical Nursing Program - 1 - 1
470 - Wabash College - 1 - 1
471 - Farmington Missouri Hospital Company - 1 - 1
472 - Relay Graduate School of Education - 1 - 1
473 - College of Menominee Nat

643 - Bethel University - 16 - 16
644 - University of North Texas Health Science Center at Fort Worth - 5 - 5
645 - St. Philip's College - 3 - 3
646 - Western Piedmont Community College - 2 - 2
647 - Caddo Kiowa Technology Center - 2 - 2
648 - University of Massachusetts Amherst - 14 - 14
649 - Montana Tech of the University of Montana - 2 - 2
650 - Stephen F Austin State University - 9 - 9
651 - VA Pittsburgh Healthcare System - 5 - 5
652 - Ashland Community and Technical College - 1 - 1
653 - Jesse Brown VA Medical Center - 2 - 2
654 - Appalachian College of Pharmacy - 2 - 2
655 - Hunter College of the City University of New York - 11 - 11
656 - Northeast Alabama Community College - 2 - 2
657 - Columbus State Community College - 4 - 4
658 - Spring Arbor University - 6 - 6
659 - Southern Methodist University - 8 - 8
660 - Edith Nourse Rogers Memorial Veterans Hospital - 4 - 4
661 - California State University - Dominguez Hills - 7 - 7
662 - Faxton-St Luke's Healthcare School of Radiol

840 - Concorde Career College - Aurora - 8 - 8
841 - VA Eastern Kansas Health Care System - 1 - 1
842 - Durham VA Medical Center - 4 - 4
843 - Ventura College - 1 - 1
844 - Bryant and Stratton College - 14 - 14
845 - Concordia University Irvine - 1 - 1
846 - Randolph-Macon College - 1 - 1
847 - College of Southern Maryland - 5 - 5
848 - Brockton Hospital - 1 - 1
849 - Benedictine College - 3 - 3
850 - Sage College of Albany - 1 - 1
851 - Mid Michigan Community College - 3 - 3
852 - Queens University of Charlotte - 5 - 5
853 - Pontifical College Josephinum - 1 - 1
854 - VA Sierra Nevada Health Care System - 2 - 2
855 - Schoolcraft College - 2 - 2
856 - Central Oregon Community College - 2 - 2
857 - Blue Ridge Community College - 1 - 1
858 - Inter American University of Puerto Rico Fajardo - 1 - 1
859 - Missouri Western State University - 6 - 6
860 - Kanawha Valley Community and Technical College - 2 - 2
861 - Philadelphia VA Medical Center - 3 - 3
862 - Rowan College at Gloucester Count

...

1800 - Joliet Junior College - 3 - 3
1801 - University of Evansville - 5 - 5
1802 - Frankford Hospital - 1 - 1
1803 - Carson-Newman University - 7 - 7
1804 - Northern State University - 2 - 2
1805 - Point Loma Nazarene University - 7 - 7
1806 - Sherman College of Chiropractic - 1 - 1
1807 - Tidewater Community College - 6 - 6
1808 - Cardinal Stritch University - 5 - 5
1809 - Western Michigan University - 20 - 20
1810 - Saddleback College - 1 - 1
1811 - Mount Aloysius College - 5 - 5
1812 - Southwestern University - 1 - 1
1813 - VA Central Western Massachusetts Healthcare System - 1 - 1
1814 - Central Maine Community College - 1 - 1
1815 - Diablo Valley College - 3 - 3
1816 - Southeastern Louisiana University - 8 - 8
1817 - Elmira College - 4 - 4
1818 - Vatterott College - Des Moines - 2 - 2
1819 - Madisonville Community College - 5 - 5
1820 - Tyler Junior College - 4 - 4
1821 - Saint Francis Medical Center College of Nursing - 3 - 3
1822 - Texas State University - 13 - 13
1823 - Carlos

2076 - Washington Adventist University - 5 - 5
2077 - The Institute for Health Education - 2 - 2
2078 - PITC Institute - 1 - 1
2079 - Baptist Health System - 2 - 2
2080 - Garden City Community College - 1 - 1
2081 - Elgin Community College - 7 - 7
2082 - Idaho State University - 27 - 27
2083 - College of Southern Nevada - 9 - 9
2084 - University of Pittsburgh - Titusville - 2 - 2
2085 - Inter American University of Puerto Rico Ponce - 4 - 4
2086 - Finlandia University - 2 - 2
2087 - Chicago State University - 7 - 7
2088 - Veterans Health Care System of the Ozarks - Mount Vernon - 2 - 2
2089 - Clarkson University - 2 - 2
2090 - Laramie County Community College - 5 - 5
2091 - Community College of Rhode Island - 10 - 10
2092 - Lesley University - 3 - 3
2093 - Sentara Hospitals - 1 - 1
2094 - Johnson C Smith University - 1 - 1
2095 - Northeastern Seminary at Roberts Wesleyan College - 3 - 3
2096 - University of North Texas - 9 - 9
2097 - Georgetown University - 11 - 11
2098 - Harrington Co

2262 - The Citadel - 1 - 1
2263 - William Paterson University of New Jersey - 7 - 7
2264 - Red Rocks Community College - 1 - 1
2265 - Branford Hall Career Institute - Branford Campus - 3 - 3
2266 - Clark Atlanta University - 1 - 1
2267 - New York Methodist Hospital, Center for Allied Health Education - 2 - 2
2268 - Western Oklahoma State College - 2 - 2
2269 - University of Oregon - 10 - 10
2270 - La Roche College - 5 - 5
2271 - Rend Lake College - 2 - 2
2272 - California Institute of the Arts - 4 - 4
2273 - Maine College of Health Professions - 2 - 2
2274 - University of Massachusetts - Worcester - 7 - 7
2275 - Mississippi Valley State University - 3 - 3
2276 - College of New Rochelle, The - 5 - 5
2277 - Colorado State University - 14 - 14
2278 - Onondaga Community College - 4 - 4
2279 - Excelsior College - 5 - 5
2280 - University of Delaware - 13 - 13
2281 - Central College - 1 - 1
2282 - Columbus Regional Hospital - 1 - 1
2283 - Lurleen B Wallace Community College - 1 - 1
2284 - Cro

2520 - Erie Community College - 8 - 8
2521 - Ogeechee Technical College - 4 - 4
2522 - University of Pennsylvania - 23 - 23
2523 - Sikeston Career and Technology Center - 2 - 2
2524 - Hostos Community College of the City University of New York - 4 - 4
2525 - Salina Area Technical College - 1 - 1
2526 - Seward County Community College - 1 - 1
2527 - University of Vermont - 13 - 13
2528 - St Vincent Catholic Medical Center New York-Brooklyn and Queens - 1 - 1
2529 - University of Great Falls - 1 - 1
2530 - University of South Carolina - Columbia - 23 - 23
2531 - Ohio State University - 45 - 45
2532 - University of Arkansas Community College-Batesville - 1 - 1
2533 - Bainbridge State College - 1 - 1
2534 - Minnesota School of Business - 12 - 12
2535 - Jones County Junior College - 2 - 2
2536 - Lindenwood University - 1 - 1
2537 - Montreat College - 1 - 1
2538 - South Mountain Montessori Teacher Education Program - 1 - 1
2539 - North Carolina Central University - 7 - 7
2540 - Graceland Uni

2741 - Blanton-Peale Institute - 1 - 1
2742 - Parkland College - 7 - 7
2743 - Southern Oklahoma Technology Center - 1 - 1
2744 - Regis College - 3 - 3
2745 - Maricopa Community Colleges - South Mountain Community College - 1 - 1
2746 - Nevada Regional Technical Center - 1 - 1
2747 - Dominican College of Blauvelt - 7 - 7
2748 - Appalachian Regional Healthcare - 1 - 1
2749 - Nicholls State University - 11 - 11
2750 - Valley City State University - 2 - 2
2751 - Bemidji State University - 2 - 2
2752 - Mohawk Valley Community College - 4 - 4
2753 - Sanford-Brown College at Jacksonville - 10 - 10
2754 - AF International School of Languages, Inc. - 1 - 1
2755 - University of Hartford - 12 - 12
2756 - Indiana University Northwest - 8 - 8
2757 - Hiwassee College - 1 - 1
2758 - Northwest Nazarene University - 4 - 4
2759 - Good Samaritan College of Nursing and Health Science - 3 - 3
2760 - Frostburg State University - 3 - 3
2761 - William Jewell College - 3 - 3
2762 - Washington Hospital - 2 - 2


In [9]:
len(txpool)

26458

In [ ]:
import time
graph = Graph(f"bolt://{hostname}:7687", auth=("neo4j", "12345678"))
tx = graph.begin()
start_time = time.time() 
[tx.create(e) for e in txpool]
print(f"Before commit: {time.time() - start_time}")
graph.commit(tx)
print(f"After commit: {time.time() - start_time}")

In [ ]:
graph.run("MATCH (n) RETURN COUNT(n)")

In [88]:
#graph.run("MATCH (n) DETACH DELETE(n)")

IndexError: index out of range